<a href="https://colab.research.google.com/github/koppalamounika/nndl/blob/main/program_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

reading data

In [20]:
data = pd.read_excel("/content/drive/MyDrive/Load_Data.xlsx",header=None)
data.head()

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2184 non-null   float64
dtypes: float64(1)
memory usage: 17.2 KB


finding null values

In [22]:
data.isnull()

,0
0,False
1,False
2,False
3,False
4,False
...,...
2179,False
2180,False
2181,False
2182,False


outliers

In [23]:
z = np.abs(stats.zscore(data))
print(z)

[[0.04910416]
 [0.08229029]
 [0.08782131]
 ...
 [0.09743692]
 [0.07253822]
 [0.03181432]]


In [24]:
df = data
q = df[0].quantile(0.99)
df[df[0] < q]
q_low = df[0].quantile(0.01)
q_hi  = df[0].quantile(0.99)

df_filtered = df[(df[0] < q_hi) & (df[0] > q_low)]
df_filtered.head()

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980


In [25]:
print(len(data))
print(len(df_filtered))
if(len(data) == len(df_filtered)):
    print('No outliners')

2184
2140


normalization

In [26]:
data = df_filtered
data_n = data.copy()
data_n = (data_n - data_n.min())/(data_n.max() - data_n.min())
data_n.head()

,0
0,0.372950
1,0.254941
2,0.235273
3,0.273574
4,0.299162


splitting data

In [27]:
train_data,test_data = train_test_split(data_n,test_size=0.1)

In [28]:
print(len(train_data))
train_data.head()

1926


,0
640,0.985120
710,0.572995
1120,0.304112
58,0.528968
138,0.804289


In [29]:
xa = []
ya = []
for i in range(len(train_data.iloc[:,0])-(7*24)):
    xa.append(train_data.iloc[i,0])
    ya.append(train_data.iloc[i+(7*24),0])

training data

In [30]:
from sympy import Symbol, Derivative, symbols
m= Symbol('m')
c= Symbol('c')
function = '0.5*(y-m*x-c)**2'
partialderiv= Derivative(function, m)
dfm = partialderiv.doit()
partialderiv= Derivative(function, c)
dfc = partialderiv.doit()
err = []
m1 = 1.0 
c1 = -1.0  
itr = 20  
GM = 0
GC = 0
ep = 10**(-8)
learning_rate = 0.1  
for i in range(0,itr):
    er = 0
    for j in range(0,len(xa)):
        er +=  ((ya[j] - (m1 * xa[j]) - c1))**2
        m = symbols('m')
        c = symbols('c')
        x = symbols('x')
        y = symbols('y')
        dfmv = dfm.subs(m, m1) 
        dfmv = dfmv.subs(c, c1) 
        dfmv = dfmv.subs(x, xa[j]) 
        dfmv = dfmv.subs(y, ya[j]) 
        dfmv = round(dfmv,2)
        dfcv = dfc.subs(c, c1) 
        dfcv = dfcv.subs(m, m1) 
        dfcv = dfcv.subs(x, xa[j]) 
        dfcv = dfcv.subs(y, ya[j]) 
        dfcv = round(dfcv,2)
        GM = GM + (dfmv**2)
        GC = GC + (dfcv**2)
        dm = (-1.0)*(learning_rate/(GM+ep)**(0.5))*dfmv
        dc = (-1.0)*(learning_rate/(GC+ep)**(0.5))*dfcv
        m1 = m1 + dm
        m1 = round(m1, 2) 
        c1 = c1 + dc
        c1 = round(c1, 2)
        er = (1/(2*len(xa)))*(er)
        err.append(er)
print("m = {} and c = {}".format(m1, c1))
print("minimum value obtained at m = {} ,c = {}".format(m1,c1))

m = 0.25 and c = 0.28
minimum value obtained at m = 0.25 ,c = 0.28


finding error for train data

In [31]:
train_data_pred = [i for i in xa]
for k in range(len(xa)):
    train_data_pred[k]=(m1*xa[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(ya,train_data_pred)
mse

0.05256023863763947

finding error for test data

In [32]:
xa = []
ya = []
for i in range(len(test_data.iloc[:,0])-24):
    xa.append(test_data.iloc[i,0])
    ya.append(test_data.iloc[i+24,0])
test_data_pred = [i for i in xa]
for k in range(len(xa)):
    test_data_pred[k]=(m1*xa[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(ya,test_data_pred)
mse

0.04906307997191567

predicting the output

In [33]:
x_max = y_max= 8841.669480
x_min = y_min = 3377.919600
x =x1 = float(input("enter load of previous hour :"))
x = (x - x_min)/(x_max - x_min)
y = m1*x + c1
y = y*(y_max - y_min) + y_min
print("load at hour based on its previous hour is : {}".format(y))

enter load of previous hour :1940
load at hour based on its previous hour is : 4547.11581388672
